In [ ]:
!pip install unidecode
!pip install import-ipynb
!pip install -U -q PyDrive
!pip install --upgrade spacy
!pip install stanza
!pip install spacy-stanza
!pip install "spacy-stanza<0.3.0"

In [ ]:
from google.colab import drive; drive.mount('/content/drive')

import pdb
import import_ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

constants_module = drive.CreateFile({'id':'...'})
constants_module.GetContentFile('constants.ipynb')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from constants import STOP_WORDS, NAMED_ENTITES

importing Jupyter notebook from constants.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
import re
from unidecode import unidecode

In [ ]:
import stanza; stanza.download('es')
from spacy_stanza import StanzaLanguage

2021-08-17 03:41:54 INFO: Downloading default packages for language: es (Spanish)...


2021-08-17 03:43:46 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
snlp = stanza.Pipeline(lang="es")
nlp = StanzaLanguage(snlp)

2021-08-17 03:44:24 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-08-17 03:44:25 INFO: Use device: cpu
2021-08-17 03:44:25 INFO: Loading: tokenize
2021-08-17 03:44:25 INFO: Loading: mwt
2021-08-17 03:44:25 INFO: Loading: pos
2021-08-17 03:44:25 INFO: Loading: lemma
2021-08-17 03:44:25 INFO: Loading: depparse
2021-08-17 03:44:25 INFO: Loading: ner
2021-08-17 03:44:27 INFO: Done loading processors!


# **Funciones para preprocesado**

In [ ]:
def process_web_links(doc):
    doc = re.sub(r"https\S+", "", doc)
    doc = re.sub(r"\S+.(\.com|\.ar|\.net|\.org|\.info|\.io|\.gov|\.edu|\.name|\.tv)", "", doc)
    doc = re.sub(r"www.\S+", "", doc)
    return doc

def process_emails(doc):
    doc = re.sub(r"[\w\.-]+@[\w\.-]+(\.[\w]+)+", "", doc)
    return doc

def process_sites_fetures(doc):
    doc = re.sub('(\w+)_(\w+)',"", doc)
    doc = re.sub(r'@[A-Za-z0-9]+',"", doc)
    doc = re.sub(r'#[A-Za-z0-9]+',"", doc)
    return doc

def process_human_expressions(doc):
    # TODO: verificar los espacios de lineas, usar regex
    doc = doc.replace(" 100% ", " muy ")
    doc = doc.replace(" x "," por ")
    doc = doc.replace(" tmb "," tambien ")
    doc = doc.replace(" zi "," si ")
    return doc

def process_generalities(doc):
    doc = re.sub(r"\W", " ", doc)
    doc = re.sub("\d+", "", doc)
    doc = re.sub(r"\s+[a-zA-Z]\s+", " ", doc)
    doc = unidecode(doc.lower())
    return doc

def process_reps(doc):
    # TODO: no elimmina las 'r' y las 'l' repetidas, lo ideal sería que elimine a partir de las tres reps
    # TODO: cuidado porque elimina las doble 'c' internas y las dobles vocales como en 'leer'
    processed_reps = re.sub(r"(?!l|r)(.)\1{1,}", r"\1", doc)
    processed_reps =  re.sub(r"\b[a-zA-Z]\b", "", processed_reps)
    return processed_reps

def process_isolated_consonants(doc):
    doc = re.sub(r"(?<!\S)(|)(?:(?![aeiou])[a-z]){2,}(?=\s|$)(?!\S)", "", doc)
    return doc

def process_k(doc):
    processed_k = re.sub(" k ", " que ", doc)
    processed_k = re.sub("ke ", "que ", processed_k)
    processed_k = re.sub("kie", "quie", processed_k)
    if r"k(.+?)":
        processed_k = re.sub(" k", " c", processed_k)
    return processed_k

def process_q(doc):
    processed_q = re.sub("q ", "que ", doc)
    processed_q = re.sub("qe ", "que ", processed_q)
    processed_q = re.sub("qie", "quie", processed_q)
    return processed_q

def process_re(doc):
    processed_re = re.sub(" re ", " muy ", doc)
    if processed_re.startswith('re '):
      processed_re = re.sub(r"^\w+", "muy ", processed_re)
    return processed_re

def process_laughts(doc):
    doc = re.sub(r"[ja]{5,}", "jaja", doc)
    doc = re.sub(r"[je]{5,}", "jeje", doc)
    doc = re.sub(r"[ji]{5,}", "jiji", doc)
    doc = re.sub(r"[jo]{5,}", "jojo", doc)
    doc = re.sub(r"[ju]{5,}", "juju", doc)
    return doc

def process_whitespaces(doc):
    doc = re.sub(r"\s+", " ", doc)
    doc = re.sub(r"(^\s+\b|(\b\ )[\.?!\s]*$)", "", doc)
    return doc

In [ ]:
def process_regex(doc):
    if not isinstance(doc, str):
      doc = str(doc)
    processed_doc = process_web_links(doc)
    processed_doc = process_emails(processed_doc)
    processed_doc = process_sites_fetures(processed_doc)
    processed_doc = process_human_expressions(processed_doc)
    processed_doc = process_generalities(processed_doc)
    processed_doc = process_reps(processed_doc)
    processed_doc = process_isolated_consonants(processed_doc)
    processed_doc = process_k(processed_doc)
    processed_doc = process_q(processed_doc)
    processed_doc = process_re(processed_doc)
    processed_doc = process_laughts(processed_doc)
    processed_doc = process_whitespaces(processed_doc)
    return processed_doc

def process_stopwords(doc, match_entities=True, del_entities=False):
    for word in doc.split():
      if word in STOP_WORDS:
        doc = re.sub(rf"(?<!\S){word}(?!\S)", "", doc)
      if match_entities:
        if word in NAMED_ENTITES:
            if del_entities:
              doc = re.sub(rf"(?<!\S){word}(?!\S)", "", doc)
            else:
                doc = re.sub(rf"(?<!\S){word}(?!\S)", "entity", doc)
    doc = process_whitespaces(doc)
    return doc

def get_stanza_lemmas(doc):
    lemmas = ""
    for sent in nlp.pipe([doc]):
      for token in sent:
        token = token.lemma_
        lemmas += token + " "
    return lemmas

In [ ]:
def get_clean_text(text, 
                   del_stops=True, 
                   match_entities=True, 
                   del_entities=False, 
                   get_lemmas=False,
                   ):
    """
    Función principal para preprocesado.

    inputs:
    text(str): texto para preprocesar.
    del_stops (bool): elimina stopwords.
    match_entities(bool): reemplaza nombres de paises, ciudades y seres humanos por "entity".
    del_entities(bool): elimina todas las palabras que hayan sido reemplazadas por "entity".
    get_lemmas(bool): obtiene lemmas de los tokens desde la librería Stanza.

    output:
    clean_text(str): texto preprocesdo.
    """
  
    clean_text = process_regex(text)

    if get_lemmas:
      print("Atención: La obtención de lemmas a través de Stanza puede demorar unos minutos...")
      clean_text = get_stanza_lemmas(clean_text)

    if del_stops:
      clean_text = process_stopwords(clean_text, match_entities=match_entities, del_entities=del_entities)
      
    clean_text = process_whitespaces(clean_text)

    return clean_text